#M1. Actividad
### H. Daniel Martínez Rodríguez A01177464
### Joaquín Aguirre de la Fuente A01177479
### Juan Carlos Martínez Zacarías A01612967

## Aspiradoras
### Instrucciones:

Dado:

* Habitación de MxN espacios.
* Número de agentes.
* Porcentaje de celdas inicialmente sucias.
* Nivel de carga máximo de los agentes. Puede ser en porcentaje o en unidades de carga.
* Tiempo máximo de ejecución.

Realiza la siguiente simulación:

* Inicializa las celdas sucias (ubicaciones aleatorias).
* Todos los agentes empiezan en la primera fila. Se puede considerar superposición, es decir, más de un agente está en la misma celda.
* Todos los agentes de limpieza tienen una carga finita de batería que se irá desgastando a medida que pasa el tiempo de forma lineal. Si la carga es cero, el agente no puede volver a moverse. Los agentes solamente se pueden cargar si, y solo si, pasan por la primera fila de la cuadrícula.
* En cada paso de tiempo:
  * Si la celda está sucia, entonces aspira.
  * Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
  * La batería se descarga una unidad.
* Se ejecuta el tiempo máximo establecido con un nivel de carga inicial igual para todos los agentes..

Deberás recopilar la siguiente información durante la ejecución:

* Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
* Porcentaje de celdas limpias después del termino de la simulación.
* Número de movimientos realizados por todos los agentes.

Analiza cómo la cantidad de agentes impacta el tiempo dedicado, así como la cantidad de movimientos realizados. Desarrollar un informe con lo observado. Incluye el diagrama de tu máquina de estados del agente.

# Frameworks/librerías a utilizar

In [253]:
%pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [254]:
import mesa

# 'Type' permite identificar la clase a la cual pertenece un objeto
# 'Callable' permite que una función esté definida con parámetros opcionales
from typing import Type, Callable

# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'MultiGrid' permite tener más de una instancia del agente en una celda
from mesa.space import MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import RandomActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación)
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Agentes
###-Robot/máquina de limpieza (CleanerAgent)
###-Basura/elementos a limpiar (TrashAgent)

In [255]:
# Clase 'CleanerAgent'
class CleanerAgent(Agent):

    battery = None
    
    # Constructor: un objeto de esta clase se inicializa con una cantidad numérica de batería, principalmente
    def __init__(self, unique_id, model, battery=None):
        super().__init__(unique_id, model)
        self.battery = battery
    
    # Función move, identifica las celdas a las que el objeto se puede mover y
    # elige un aleatoriamente
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    # Función clean, obtiene el contenido de la celda en la que se encuentra
    # el objeto y, si encuentra un TrashAgent "activo", lo "desactiva"     
    def clean(self):
        current_position = []
        current_position.append(self.pos)
        
        for obj in self.model.grid.get_cell_list_contents(current_position):
            if isinstance(obj, TrashAgent):
                if obj.status == 1:
                    obj.status = 0
                    return True
        return False

    # Función step, si el objeto cuenta con batería, llama a la función move y
    # reduce su batería en una unidad    
    def step(self):
        if self.battery > 0:
          if self.clean() == False:
              self.move()
          self.battery = self.battery - 1
          x, y = self.pos
          if x == 0:
              self.battery += 1
        
# Clase TrashAgent        
class TrashAgent(Agent):

    #Constructor: Una instancia de esta clase recibe un estado inicial de 1
    def __init__(self, unique_id, model):     
        super().__init__(unique_id, model)
        self.status = 1
        self.next_state = None

# Creación del modelo

In [256]:
# Clase RandomActivationByTypeFiltered
class RandomActivationByTypeFiltered(mesa.time.RandomActivationByType):

    # Función get_type_count, realiza el conteo del tipo de agente indicado, con
    # o sin condiciones especiales
    def get_type_count(
        self,
        type_class: Type[mesa.Agent],
        filter_func: Callable[[mesa.Agent], bool] = None,
    ) -> int:

        count = 0
        for agent in self.agents_by_type[type_class].values():
            if filter_func is None or filter_func(agent):
                count += 1
        return count

# Clase TrashModel
class CleanTrashModel(Model):

    # Constructor
    def __init__(
        # Parámetros por defecto
        self, 
        width=10,
        height=10,
        battery=100,
        initial_cleaners=10,
        trash_percentage=0.5
    ):
        # Asignación de parámetros
        self.width = width
        self.height = height
        self.battery = battery
        self.initial_cleaners = initial_cleaners
        self.trash_percentage = trash_percentage
        self.num_agents = width * height

        self.grid = MultiGrid(width, height, False)
        self.schedule = RandomActivationByTypeFiltered(self)
        self.clean = 0
        self.iterations = 0

        # Creación aleatoria de celdas "sucias", con base en el porcentaje indicado
        for (content, x, y) in self.grid.coord_iter():
            if self.random.random() < self.trash_percentage:
                a = TrashAgent((x, y), self)
                self.grid.place_agent(a, (x, y))
                self.schedule.add(a)

        # Creación de "robots/máquinas" con base en la cantidad indicada, aleatoriamente
        # dentro de la primera fila y considerando la superposición        
        for i in range(self.initial_cleaners):
            x = 0
            y = self.random.randrange(self.width)
            battery = battery
            a = CleanerAgent(i, self, battery)
            self.grid.place_agent(a, (x,y))
            self.schedule.add(a)
            
        # Recolección del grid para su graficación y de la cantidad de celdas "sucias"    
        self.datacollector = DataCollector(
            {
                "Grid": self.get_grid,
                "Trash_Cells": lambda m: m.schedule.get_type_count(
                    TrashAgent, lambda x: x.status
                )

            }
        )

        self.running = True
        self.datacollector.collect(self)
    
    # Función step. En cada paso el colector toma la información que se definió
    # y almacena el grid para luego graficarlo.
    def step(self):
        
        self.datacollector.collect(self)
        self.schedule.step()

        grid = np.zeros((model.grid.width, model.grid.height))
        for cell in model.grid.coord_iter():
          cell_content, x, y = cell
          for obj in cell_content:
            if isinstance(obj, TrashAgent):
              if obj.status == 0:
                self.clean = self.clean + 1
        
        if self.clean < self.num_agents:
          self.clean = 0
          self.iterations = self.iterations + 1
          return True

        else:
          print("Iteraciones para que termine de limpiar: ", self.iterations)
          return False

    # Función get_grid. Obtiene la información del grid.
    def get_grid(model):
        grid = np.zeros((model.grid.width, model.grid.height))
        for cell in model.grid.coord_iter():
          cell_content, x, y = cell
          for obj in cell_content:
            if isinstance(obj, CleanerAgent):
              grid[x][y] = 2
            elif isinstance(obj, TrashAgent):
              grid[x][y] = obj.status
        return grid

# Batch Run para análisis del modelo

In [265]:
# Definición de parámetros (10 "robots" iniciales)
params = {"width": 10, "height": 10, "battery": 100, "initial_cleaners": 10, "trash_percentage": 0.5}

# Ejecución con 3 iteraciones y un máximo de 75 pasos
results = mesa.batch_run(
    CleanTrashModel,
    parameters=params,
    iterations=3,
    max_steps=75,
    number_processes=1,
    data_collection_period=1,
    display_progress=False,
)

# Almacenamiento de resultados
results_df = pd.DataFrame(results)

# Impresión de cada iteración
for i in range(0, 3, 1):
    results_df = pd.DataFrame(results)
    one_episode_dirt = results_df[(results_df.iteration == i)]
    print(
        one_episode_dirt.to_string(
            index=False, columns=["iteration", "Step", "initial_cleaners", "Trash_Cells"], max_rows=20
        )
    )

 iteration  Step  initial_cleaners  Trash_Cells
         0     0                10           47
         0     1                10           47
         0     2                10           44
         0     3                10           43
         0     4                10           43
         0     5                10           40
         0     6                10           37
         0     7                10           36
         0     8                10           33
         0     9                10           33
       ...   ...               ...          ...
         0    66                10            0
         0    67                10            0
         0    68                10            0
         0    69                10            0
         0    70                10            0
         0    71                10            0
         0    72                10            0
         0    73                10            0
         0    74                10      

In [266]:
# Definición de parámetros (20 "robots" iniciales)
params = {"width": 10, "height": 10, "battery": 100, "initial_cleaners": 20, "trash_percentage": 0.5}

# Ejecución con 3 iteraciones y un máximo de 75 pasos
results = mesa.batch_run(
    CleanTrashModel,
    parameters=params,
    iterations=3,
    max_steps=75,
    number_processes=1,
    data_collection_period=1,
    display_progress=False,
)

# Almacenamiento de resultados
results_df = pd.DataFrame(results)

# Impresión de cada iteración
for i in range(0, 3, 1):
    results_df = pd.DataFrame(results)
    one_episode_dirt = results_df[(results_df.iteration == i)]
    print(
        one_episode_dirt.to_string(
            index=False, columns=["iteration", "Step", "initial_cleaners", "Trash_Cells"], max_rows=80
        )
    )

 iteration  Step  initial_cleaners  Trash_Cells
         0     0                20           42
         0     1                20           42
         0     2                20           38
         0     3                20           36
         0     4                20           34
         0     5                20           33
         0     6                20           29
         0     7                20           27
         0     8                20           26
         0     9                20           25
         0    10                20           24
         0    11                20           22
         0    12                20           20
         0    13                20           20
         0    14                20           18
         0    15                20           17
         0    16                20           15
         0    17                20           15
         0    18                20           14
         0    19                20      

In [267]:
# Definición de parámetros (30 "robots" iniciales)
params = {"width": 10, "height": 10, "battery": 100, "initial_cleaners": 30, "trash_percentage": 0.5}

# Ejecución con 3 iteraciones y un máximo de 75 pasos
results = mesa.batch_run(
    CleanTrashModel,
    parameters=params,
    iterations=3,
    max_steps=75,
    number_processes=1,
    data_collection_period=1,
    display_progress=False,
)

# Almacenamiento de resultados
results_df = pd.DataFrame(results)

# Impresión de cada iteración
for i in range(0, 3, 1):
    results_df = pd.DataFrame(results)
    one_episode_dirt = results_df[(results_df.iteration == i)]
    print(
        one_episode_dirt.to_string(
            index=False, columns=["iteration", "Step", "initial_cleaners", "Trash_Cells"], max_rows=20
        )
    )

 iteration  Step  initial_cleaners  Trash_Cells
         0     0                30           45
         0     1                30           45
         0     2                30           41
         0     3                30           38
         0     4                30           37
         0     5                30           36
         0     6                30           33
         0     7                30           32
         0     8                30           30
         0     9                30           27
       ...   ...               ...          ...
         0    66                30            0
         0    67                30            0
         0    68                30            0
         0    69                30            0
         0    70                30            0
         0    71                30            0
         0    72                30            0
         0    73                30            0
         0    74                30      

# Visualización del modelo

In [260]:
GRID_SIZE = 10

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 155

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = CleanTrashModel(GRID_SIZE, GRID_SIZE)
for i in range(NUM_GENERATIONS):
    if model.step() == False:
      NUM_GENERATIONS = i + 1
      break

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.087837


In [261]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [262]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    

In [263]:
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [264]:
anim

# Conclusiones
Sin duda alguna, la presente actividad no fue más que una demostración de todo aquello que se puede realizar utilizando una modelación basada en agentes, como en este caso fue la situación determinada por un espacio constituido por celdas, siendo un porcentaje de ellas (inicialmente el 50%) celdas con una condición en particular: sucias, para lo cual es necesario el diseño e implementación de robots de limpieza con parámetros variables, como su nivel inicial de batería y su misma cantidad de instancias que, partiendo desde el inicio del espacio, y con la posibilidad de recargarse en este punto, tienen la labor de limpiarlo.
Es por ello por lo que al realizar varias ejecuciones del modelo se pudo observar cómo es que la cantidad de robots impacta directamente al tiempo requerido para llegar a una misma cantidad de celdas limpias con respecto a la cantidad inicial, puesto que con 30 robots, tomando en cuenta un máximo de 75 pasos, el promedio de pasos requeridos para solamente tener una celda sucia, en tres simulaciones, fue de 49 pasos, mientras que para 20 robots este valor fue de 57 pasos. Finalmente, como era de esperarse, en promedio, 10 robots se quedaron en 4 celdas sucias en el paso 75.
Si bien hay diferencias, estas no son demasiado grandes, por lo que a futuro queda la posibilidad de implementar un "pensamiento" más inteligente con la finalidad de optimizar el modelo y encontrar un equilibrio entre la cantidad de robots requeridos para limpiar todo o casi todo un espacio determinado.